In [1]:
import lightgbm as lgb
import os
import pandas as pd
import numpy as np
from tqdm import tqdm, tqdm_notebook, tnrange
import gc
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import roc_auc_score
from scipy.sparse import hstack,vstack
import time
import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_pickle('./data/data_all.pickle')

In [ ]:
def train_test_split(data):
    origin_feature   = ['用户二分类特征A', '用户二分类特征B', '用户二分类特征C','用户二分类特征D', '用户二分类特征E','用户的盐值分数']
    lbl_enc_feat = ['性别_lbl_enc','访问频率_lbl_enc','用户分类特征A_lbl_enc', '用户分类特征B_lbl_enc', '用户分类特征C_lbl_enc', '用户分类特征D_lbl_enc','用户分类特征E_lbl_enc']
    user_ques_svd    = ['问题绑定的话题ID_svd_{}'.format(i) for i in range(1,31)]
    user_ques_watched_svd  = ['用户关注的话题_svd_{}'.format(i) for i in range(1,11)]
    user_ques_fav_svd  = ['user_ques_fav_svd_{}'.format(i) for i in range(1,11)]
    nn_stack = ['w2v_sum_nn']

    test_feat = ['问题邀请用户_counts','用户被邀请问题_counts','用户的盐值分数_max',
                 '用户的盐值分数_min','用户的盐值分数_std','用户的盐值分数_mean','invite_answer_gap','邀请问题创建时间_gap',
                '用户关注的话题_len','用户感兴趣的话题_len','问题绑定的话题ID_len','prev_ans_ques_title_sim_min', 'prev_ans_ques_title_sim_max',
           'prev_ans_ques_title_sim_mean', 'prev_ans_ques_title_sim_std']
    
    new_feat = ['用户ID_last_expo','用户ID_next_expo','问题ID_last_expo','问题ID_next_expo','问题创建时间_H','邀请创建时间_H']
    
    new_feat_II = ['用户_问题IDs_svd_{}'.format(i) for i in range(1,11)]+['问题_用户IDs_svd_{}'.format(i) for i in range(1,11)]+['lstm_enc_feat']+\
     ['邀请创建时间_mean','邀请创建时间_max','邀请创建时间_min','邀请创建时间_std','问题创建时间_mean','问题创建时间_max','问题创建时间_min',
      '问题创建时间_std','用户ID曾经回答数','用户问题ID_count','用户问题话题相同个数','用户感兴趣问题话题相同个数']
      
    new_feat_III = ['问题_用户感兴趣_topic_sim','问题_用户关注_topic_sim','用户关注_感兴趣_topic_sim']
    
    new_feat_IV = ['回答是否被标优_count', '回答是否被推荐_count' , '是否包含图片_count' ,
                   '是否包含视频_count','回答字数_mean' ,'点赞数_mean', '取赞数_mean' ,'评论数_mean' ,
                   '收藏数_mean', '感谢数_mean' ,'举报数_mean', '没有帮助数_mean' ,'反对数_mean']
    
    new_feat_V = ['回答字数_sum' ,'点赞数_sum', '取赞数_sum' ,'评论数_sum' ,'收藏数_sum', '感谢数_sum' ,
                  '举报数_sum', '没有帮助数_sum' ,'反对数_sum']

    new_feat_VI = ['问题标题_曾经回答_SW_sim','问题描述_曾经回答_SW_sim','问题标题_曾经回答_W_sim','问题描述_曾经回答_W_sim','times_mean',
                   'times_max','times_min','times_std','Hs_mean','Hs_max','Hs_min','Hs_std']
    
    new_feat_VII = ['问题描述_曾经回答_Topic_sim','prev_topic_sims_min','prev_topic_sims_max','prev_topic_sims_mean','prev_topic_sims_std']
    feat_I =['question_curr_expo']
    feat_II = ['prev_ans_times_min_gap', 'prev_ans_times_mean_gap', 'qtime_std','qtime_mean', 'utime_std', 'utime_mean', 'iweek', 'qlast_itime_gap','qllast_itime_gap', 'qlllast_itime_gap', 'qnext_itime_gap','qnnext_itime_gap', 'qnnnext_itime_gap', 'ulast_itime_gap','ullast_itime_gap', 'ulllast_itime_gap', 'unext_itime_gap','unnext_itime_gap', 'unnnext_itime_gap']
    ques_len_stat_feat = ['问题标题_len', '问题描述_len', '问题描述_len-问题标题_len', '问题标题_W_len', '问题描述_W_len','问题描述_len-问题标题_len_W', '问题标题和描述的交集个数', '问题标题和描述的交集个数_W', '问题标题和描述的交集个数/问题标题_len', '问题标题和描述的交集个数/问题描述_len', '问题标题和描述的交集个数_W/问题标题_len', '问题标题和描述的交集个数_W/问题描述_len', '编辑距离',
       '前一个词语是否相同', '前两个词语是否相同', '前三个词语是否相同', '前一个词语是否相同_W', '前两个词语是否相同_W','前三个词语是否相同_W', '第一个词语在标题里面出现位置', '第二个词语在标题里面出现位置', '第三个词语在标题里面出现位置','第一感兴趣在问题绑定话题里面出现位置', '第二感兴趣在问题绑定话题里面出现位置', '第三感兴趣在问题绑定话题里面出现位置', '问题话题编辑距离',
       '问题标题_len_mean', '问题描述_len_mean', '问题标题_W_len_mean', '问题描述_W_len_mean'] 
    new_feat_VIII = feat_I + feat_II 
    #******* Feature sum***********#
    feature  =    origin_feature + lbl_enc_feat + user_ques_svd + user_ques_watched_svd + user_ques_fav_svd + nn_stack +\
                   test_feat + new_feat + new_feat_II + new_feat_III + new_feat_IV + new_feat_V + new_feat_VI+\
                  new_feat_VII + new_feat_VIII + ques_len_stat_feat
    
    #*********************************# 
    
    test_index = np.isnan(data['邀请是否被回答'])
    train_index = ~test_index
    train_x = data[train_index][feature] 
    train_y = data[train_index]['邀请是否被回答']
    test_x  = data[test_index][feature]
    train_x.columns = ['col_{}'.format(i) for i in range(210)]
    test_x.columns = ['col_{}'.format(i) for i in range(210)]
    return train_x, train_y, test_x,feature
    
train_x, train_y, test_x,feature = train_test_split(data)
del data
gc.collect()
print('All features: train shape {}, test shape {}'.format(train_x.shape, test_x.shape))
print(len(feature))

In [ ]:
tic = time.time()
preds = np.zeros((test_x.shape[0], 2))
scores = []

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for index, (tr_idx, va_idx) in enumerate(kfold.split(train_x, train_y)):
    print('*' * 30)
    X_train, y_train, X_valid, y_valid = train_x.iloc[tr_idx], train_y.iloc[tr_idx], train_x.iloc[va_idx], train_y.iloc[va_idx]
    lgb_model = lgb.LGBMClassifier(boosting_type='gbdt', num_leaves=64, reg_alpha=0.1, reg_lambda=10,
                                max_depth=-1, n_estimators=100000, objective='binary', metrics='auc', 
                                bagging_fraction=0.8, is_unbalance=False, bagging_freq=5, min_child_samples=80, 
                                feature_fraction=0.8, learning_rate=0.1, random_state=42, n_jobs=16,
                                )
    lgb_model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], eval_metric='auc',verbose=100, early_stopping_rounds=300)

    score = lgb_model.best_score_['valid_0']['auc']
    scores.append(score)
    print('fold %d round %d : score: %.6f | mean score %.6f' % (index+1, lgb_model.best_iteration_, score,np.mean(scores))) 
    pred = lgb_model.predict_proba(test_x)
    preds += pred  #mean score 0.8935
    np.save('./out/preds_lgb_{}.npy'.format(index+1), pred)
    del lgb_model
    del X_train, y_train, X_valid, y_valid
    gc.collect()

toc = time.time()
print("Wall time: %d s" % (toc-tic))

In [ ]:
invite_info_evaluate = pd.read_table('../data/data_set_0926/invite_info_evaluate_2_0926.txt',header=None)
invite_info_evaluate.columns =  ['问题ID','用户ID','邀请创建时间']
result = invite_info_evaluate[['问题ID','用户ID','邀请创建时间']]
result['result'] = preds[:, 1] / 5
result.to_csv('./out/result_lgb.txt', sep='\t', index=False, header=False)